# Cashflow Computation

In [0]:
public static double[] GetConstantValues(this double initialValue, int shift, int length){

    var constant = new double[shift + length];
    
    for (var i = 0; i < length; i++)
        constant[shift + i] =  initialValue;
        
    return constant;
}

## Compute Linear Cashflow

In [0]:
public static double[] GetLinearValues(this double initialValue, int shift, int length, double[] values){

    // The cashflow start after "shift" periods. 
    // The cashflow length is "Lenght" periods.
    // The cashflow is the accumulation of "initialValue" + "value[i]"

    var linear = new double[shift + length];
    
    for (var i = 0; i < length; i++)
        linear[shift + i] =  initialValue + values.Take(i + 1).Aggregate((x, y) => x + y);
        
    return linear;
}

## Compute Compounded Cashflow

In [0]:
public static double[] GetCompoundValues(this double initialValue, int shift, int length, double[] values){
    
    var compound = new double[shift + length];

    for (var i = 0; i < length; i++)
        compound[shift + i] = initialValue * values.Take(i + 1).Select(x => 1 + x).Aggregate((x, y) => x * y);
             
    return compound;
}

## Compute Bounded Cashflow

In [0]:
public static double[] GetBoundedCashflow(this int shift, int length, double min, double max){
    var bounded = new double[length];
    var random = new Random(); // is this random enough?

    for (var i = shift; i < length; i++)
        bounded[i] = random.NextDouble() * (max - min) + min;
               
    return bounded;
}

## Correlate Risk Cashflows
### Two risks
Some cashflows for Risk Drivers in the Solvency framework should allow for correlation between them. This is done by using a Correlation factor. 
$$
\text{Correlated Value} = \sqrt{r_1^2 + r_2^2 + 2 \cdot a \cdot b \cdot \text{Corr}(r_1,r_2)}
$$

where 
- $$r_1$$ and $$r_2$$ represent risk measure of 2 correlated risks
- $$Corr(r_1, r_2)$$ is correlation factor between such 2 risk measures
  
The total value, *in addition* to simple aggregation of *r_1* an *r_2* is expressed by
$$
\text{Correlation Effect} = \text{Correlated Value} - r_1 - r_2
$$

Note, if both risks are the same, the correlation is assumed to be 1, and the above formula for Correlation Effect simlifies to 0. Thus, there is no need to provide correlation factors of 1.

### Multiple risks
$$
\text{Correlated Value} = \sqrt{\sum_{ij} r_i \cdot r_j \cdot \text{Corr}(r_i,r_j)}
$$

$$
\text{Correlation Effect} = \text{Correlated Value} - \sum_{i}r_i
$$

Source: [Eiopa CEIOPS Correlation Parameters](https://register.eiopa.europa.eu/CEIOPS-Archive/Documents/Advices/CEIOPS-L2-Advice-Correlation-Parameters.pdf)

In [0]:
public static double Correlate(double[] r1, double[] r2, double[,] correlation){

    if (r1.Length != r2.Length)
        throw new System.Exception("Vectors of unequal lenghts!");

   return Math.Sqrt(r1.Select((x,i) => r2.Select((y,j) => x * y * correlation[i,j] ).Sum()).Sum());

}